In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]=" "

In [2]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
sess = tf.Session()

In [3]:
from loss import *

In [4]:
vmax = 255

In [5]:
a = np.random.randint(0, 120, [6, 12, 12, 6])*1.0#tf.random.uniform([6, 320, 256, 12], maxval=vmax+1, dtype='int32')
b = np.random.randint(0, 120, [6, 12, 12, 6])*1.0#tf.random.uniform([6, 320, 256, 12], maxval=vmax+1, dtype='int32')

In [9]:
i = tf.constant(0)

ph1 = tf.placeholder(shape=[None, None, None, None], dtype=tf.float64)
ph2 = tf.placeholder(shape=[None, None, None, None], dtype=tf.float64)

output = tf.map_fn(lambda x : tf.image.ssim(x[0], x[1], tf.reduce_max(x[0])), elems=(ph1, ph2), dtype=(tf.float64, tf.float64))


ValueError: The two structures don't have the same nested structure.

First structure: type=tuple str=(tf.float64, tf.float64)

Second structure: type=Tensor str=Tensor("map_1/while/Mean_2:0", shape=(), dtype=float32)

More specifically: Substructure "type=tuple str=(tf.float64, tf.float64)" is a sequence, while substructure "type=Tensor str=Tensor("map_1/while/Mean_2:0", shape=(), dtype=float32)" is not
Entire first structure:
(., .)
Entire second structure:
.

In [ ]:
def Custom_SSIM(y_true, y_pred):
    """
    y_true : [batch, height, width, channel]
    y_pred : [batch, height, width, channel]
    """
    # b, h, w, c = tf.shape(y_true)
    #print(tf.shape(y_true))
    
    # [b, h, w, c] -> [b*c, h, w]
    tmp_true = tf.reshape(tf.transpose(y_true, [0, 3, 1, 2]),
                          [tf.shape(y_true)[0]*tf.shape(y_true)[-1], tf.shape(y_true)[1], tf.shape(y_true)[2], 1])
    tmp_pred = tf.reshape(tf.transpose(y_pred, [0, 3, 1, 2]), 
                          [tf.shape(y_true)[0]*tf.shape(y_true)[-1], tf.shape(y_true)[1], tf.shape(y_true)[2], 1])
    true_max = tf.reduce_max(y_true)
    
    
#     f1 = lambda: tf.constant(1)
#     f2 = lambda: tf.constant(255)
#     f3 = lambda: tf.constant(65535)
#     max_val = tf.case({tf.greater(true_max, 255) : f3, tf.greater(true_max, 1) : f2},
#                       default=f1, exclusive=False)
    
    
    output = tf.map_fn(lambda x : tf.image.ssim(x[0], x[1], tf.reduce_max(x[0])), elems=(tmp_true, tmp_pred), dtype=tf.float64)
    #output = tf.image.ssim(tmp_true, tmp_pred, max_val)
    #output = tf.reshape(output, [tf.shape(y_true)[0], tf.shape(y_true)[-1]])
    return output#tf.reduce_mean(output, axis=1)


In [ ]:
sess.run(r)

In [ ]:
ph1 = tf.placeholder(shape=[None, None, None, None], dtype=tf.float64)
ph2 = tf.placeholder(shape=[None, None, None, None], dtype=tf.float64)

#ssim = Custom_SSIM(ph1, ph2)
test = tf.map_fn(lambda x : tf.image.ssim(x[0], x[1], tf.reduce_max(x[0])), elems=(ph1, ph2))

In [ ]:
sess_test = sess.run(test, feed_dict={ph1:a, ph2:b})

In [ ]:
def mutual_information_np(hist2d):
    pxy = hist2d / np.sum(hist2d)
    px = np.sum(pxy, axis=1)
    py = np.sum(pxy, axis=0)
    px_py = px[:, None] * py[None, :]
    nzs = pxy > 0
    return np.sum(pxy[nzs] * np.log(pxy[nzs] / px_py[nzs]))

In [ ]:
def mutual_information_single(hist2d):
    tmp = tf.cast(hist2d, dtype='float64')
    pxy = tmp / tf.reduce_sum(tmp)
    px = tf.reduce_sum(pxy, axis=1)
    py = tf.reduce_sum(pxy, axis=0)
    px_py = px[:, None] * py[None, :]
    nzs = tf.greater(pxy, 0)
    return tf.reduce_sum(tf.boolean_mask(pxy, nzs) * tf.log(tf.boolean_mask(pxy, nzs) / tf.boolean_mask(px_py, nzs)))


In [ ]:
def tf_joint_histogram(y_true, y_pred):
    """
    y_true : [batch, height, width, channel]
    y_pred : [batch, height, width, channel]
    """
    
    vmax = 255
    
    b, h, w, c = y_true.get_shape()
    
    # [batch, height, width, channel]
    # -> [batch, height * width, channel]
    # -> [batch, channel, height * width]
    
    flat_true = tf.transpose(tf.reshape(y_true, [b, h*w, c]), [0, 2, 1])
    flat_true = tf.reshape(flat_true, [b*c, h*w])
    flat_pred = tf.transpose(tf.reshape(y_pred, [b, h*w, c]), [0, 2, 1])
    flat_pred = tf.reshape(flat_pred, [b*c, h*w])
    
    output = (flat_pred * (vmax+1)) + (flat_true+1)
    # [b*c, 65536]
    output = tf.map_fn(lambda x : tf.histogram_fixed_width(x, value_range=[1, (vmax+1)**2], nbins=(vmax+1)**2), output)
    # [b, c, 256, 256] -> [b, 256, 256, c]
    output = tf.transpose(tf.reshape(output, [b, c, vmax+1, vmax+1]), [0, 2, 3, 1])
    return output, y_true, y_pred

In [ ]:
def mutual_information(y_true, y_pred):
    
    # [b, 256, 256, c]
    joint_histogram, _, _ = tf_joint_histogram(y_true, y_pred)
    b, h, w, c = joint_histogram.get_shape()
    
    # [b*c, 256, 256]
    joint_histogram = tf.reshape(tf.transpose(joint_histogram, [0, 3, 1, 2]), [b*c, h, w])
    
    output = tf.map_fn(lambda x : mutual_information_single(x), joint_histogram, dtype=tf.float64)
    output = tf.reshape(output, [b, c])
    return output, y_true, y_pred, joint_histogram

In [ ]:
test, test_a, test_b, test_hist2d = sess.run(mutual_information(a, b))
hist2d, _, _ = np.histogram2d(test_a[0,...,0].ravel(), test_b[0,...,0].ravel(), bins=vmax+1, range=[[0,vmax], [0,vmax]])


In [ ]:
print(test[0,0])
print(mutual_information_np(hist2d))